# 09 - comissões por periodos

### extract + tranform (definindo dados e juntando dados )

In [1]:
import os,sys
current_dir = os.getcwd()
from datetime import datetime
# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)
import pandas as pd
from Scripts.Reads import ReadRaw, ReadConfigs
from Scripts.Padronizes import PadronizeProdutos
os.chdir("..")

ano, mes = ReadConfigs()

In [2]:
# -- 1 - defino as coluas de nossa df bruta e trago as df necessrias para o relatorio 
colunas = ['Clie Cgc Cpf','Dt. emissão', 'Nr. nota', 'Cd Vendedor', 'Vl Total Nota', 'comissao', 'total']
df_comission_raw = pd.DataFrame(columns=colunas)
df_vendedores = pd.read_excel(r"C:\Users\ADMIN\Desktop\kaue\py\faturamento e bi mes a mes\Data\BRUTOS-GERAIS\VENDEDORES\sh-vendedores_final.xlsx") # para pegar o vendedor
df_clientes = pd.read_excel(r"C:\Users\ADMIN\Desktop\kaue\py\faturamento e bi mes a mes\Data\BRUTOS-GERAIS\CLIENTES\sh-clientes_final.xlsx") # para pegar o cliente
df_pedidos, _ = ReadRaw(["sh-pedidos_final.xlsx"]) # para pegar os pedidos
df_nfs, _= ReadRaw(["sh-nfs_final.xlsx"])

# -- 1 - atribuindo infos das nfs 
df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)

mes2 = mes + 1

# -- 3 - fazer um semi join (pega apegas nfs que tivrem com cd vendedor) - por preucação
df_comission_raw = df_comission_raw[df_comission_raw['Cd Vendedor'].isin(df_vendedores['Cd Vendedor'])] #  pandas não tem suporte direto para semi join o jeito a epanas filtrar oque e muito pratico!

# -- 4 - fazer merge para epgar o nome dos vendedores pelo cod, posteriormente pegar o supervisor e aliquota tambem 
df_comission_raw = df_comission_raw.merge(df_vendedores[['Cd Vendedor', 'Nm Vendedor','Nm supervisor', 'aliquota']], 
                             on='Cd Vendedor', how='left')

# -- 5 - fazer merge para pegar o nome do cliente (left join em duas colunas basicamente..)
df_comission_raw = df_comission_raw.merge(df_clientes[['Clie Cgc Cpf', 'Descrição']],
                      on='Clie Cgc Cpf', how='left')

# -- 6 - merge para pegar o valor do pedido
df_comission_raw = df_comission_raw.merge(df_pedidos[['Nr Pedido', 'Vl venda']],
                      on='Nr Pedido', how='left')

# -- 7 -  definindo comissão a ser recebida 
df_comission_raw['comissao'] = df_comission_raw['Vl Total Nota']*(df_comission_raw['aliquota']/100)

# -- 8 - definindo clientes novos
df_clientes_novos = df_clientes[df_clientes['Dt Cadastro'] > datetime(ano, mes, 1)]
df_clientes_novos['cliente_nv'] = 20

# -- 9 - fazendo merge para atribuir os clientes novos 
df_comission_raw = df_comission_raw.merge(df_clientes_novos[['Clie Cgc Cpf', 'cliente_nv']],
                      on='Clie Cgc Cpf', how='left', suffixes=('_raw','_new'))

# -- 10 - definindo comissão total 
df_comission_raw['total'] = df_comission_raw['cliente_nv'].fillna(0)+df_comission_raw['comissao'].fillna(0)

# -- 11 - atribuindo apenas informações necessarias
df_comission_raw = df_comission_raw[['Nm supervisor', 'Nm Vendedor','Dt. emissão', 'Nr. nota','Descrição','Vl venda', 'Vl Total Nota','aliquota','comissao','cliente_nv','total']]


# --- 12 - susbtituindo os supervisores vazios para sem supervisor 
df_comission_raw['Nm supervisor'] = df_comission_raw['Nm supervisor'].replace(['', ' ', 'nan', None], 'SEM SUPERVISOR')

# -- 13 - Ordenando para gerar linha linha excel
df_detalhes = df_comission_raw.sort_values(by=['Nm supervisor', 'Nm Vendedor', 'Dt. emissão']) 
df_detalhes['comissao'] = df_detalhes['comissao'].round(2)
#-- 14 - define os subtotais por vendedor 
subtotal_vendedor = df_comission_raw.groupby(['Nm supervisor', 'Nm Vendedor']).agg(
    total_venda=('Vl venda', 'sum'),
    total_faturado=('Vl Total Nota', 'sum'),
    total_comissao=('comissao', 'sum'),
    total_cliente_nv=('cliente_nv', 'sum'),
    total_total=('total', 'sum')
).reset_index()

# arredonda tudo para 2 casas
subtotal_vendedor[['total_venda', 'total_faturado', 'total_comissao', 'total_cliente_nv', 'total_total']] = \
    subtotal_vendedor[['total_venda', 'total_faturado', 'total_comissao', 'total_cliente_nv', 'total_total']].round(2)

# -- 15 - ve se o supervisor tambem e vendedor para na hora de gara os 3% do supervisor nao incluir se ele for eh_vendedor tiver true 

# -- 16 - Cria base com todos os supervisores únicos
todos_supervisores = df_comission_raw['Nm supervisor'].dropna().unique()
todos_supervisores = todos_supervisores[todos_supervisores != 'SEM SUPERVISOR']
resumo_supervisor = pd.DataFrame(index=todos_supervisores)

# -- 16.1 - Extrai subtotais de supervisores que também atuaram como vendedores
totais_supervisor = subtotal_vendedor[
    subtotal_vendedor['Nm supervisor'] == subtotal_vendedor['Nm Vendedor']
][['Nm supervisor', 'total_comissao', 'total_cliente_nv']].set_index('Nm supervisor')

# Junta os totais no resumo (pode ter NaN depois)
resumo_supervisor = resumo_supervisor.join(totais_supervisor, how='left')

# -- 17 - Define o faturamento dos subordinados para cálculo dos 3%
faturamento_subordinados = df_comission_raw[
    # Regra 1: vendedor diferente do supervisor
    (df_comission_raw['Nm supervisor'] != df_comission_raw['Nm Vendedor']) |
    # Regra 2: exceção — FLAVIO E LILIAN pegam tudo
    (df_comission_raw['Nm Vendedor'] == 'FLAVIO E LILIAN')
].groupby('Nm supervisor')['Vl Total Nota'].sum().rename("faturamento_subordinados")

# Calcula os 3%
comissao_extra = faturamento_subordinados * 0.03
comissao_extra.name = 'comissao_extra_3pct'

# Junta no resumo
resumo_supervisor = resumo_supervisor.join(comissao_extra, how='left')


# -- 18 - Finaliza o resumo preenchendo nulos e somando
resumo_supervisor = resumo_supervisor.fillna(0)

resumo_supervisor['comissao_final'] = (
    resumo_supervisor['total_comissao'] +
    resumo_supervisor['comissao_extra_3pct']
)
colunas_resumo = ['total_comissao', 'comissao_extra_3pct', 'comissao_final']
resumo_supervisor[colunas_resumo] = resumo_supervisor[colunas_resumo].round(2)

# -- 19 - separa em duas df: sem e com supervisores, pois no relatorios tem estruturas diferentes.. 
df_com_supervisor = df_detalhes[
    (df_detalhes['Nm supervisor'].notna()) & 
    (df_detalhes['Nm supervisor'] != 'SEM SUPERVISOR')
]
df_sem_supervisor = df_detalhes[
    df_detalhes['Nm supervisor'] == 'SEM SUPERVISOR'
]

# -- 20 - tranforma adta em string para evitar complicações na hora de gerar o relatorio


colunas_monetarias = ['Vl venda', 'Vl Total Nota', 'comissao', 'cliente_nv', 'total']
for col in colunas_monetarias:
    df_com_supervisor[col] = pd.to_numeric(df_com_supervisor[col], errors='coerce').round(2)
    df_sem_supervisor[col] = pd.to_numeric(df_sem_supervisor[col], errors='coerce').round(2)
    


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6924\1836768043.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_comission_raw = pd.concat([df_comission_raw,df_nfs], axis=0)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6924\1836768043.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clientes_novos['cliente_nv'] = 20
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_6924\1836768043.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [3]:
df_comission_raw.head()

,Nm supervisor,Nm Vendedor,Dt. emissão,Nr. nota,Descrição,Vl venda,Vl Total Nota,aliquota,comissao,cliente_nv,total
0,ALDO,ALDO,2025-06-06,7313,PALHACINHO MODAS LTDA ( EDSON),1059.80,1059.80,7,74.1860,NaN,74.1860
1,ALDO,ALDO,2025-06-13,7323,PALHACINHO MODAS LTDA ( EDSON),1839.20,1839.20,7,128.7440,NaN,128.7440
2,ALDO,ALDO,2025-06-27,7344,PEQUENA REALEZA TUPA LTDA,1458.74,1458.74,7,102.1118,NaN,102.1118
3,MELISSA SACRAMENTO,REGINA-HERCULES,2025-06-05,7309,DEUSELINA BERNARDES DE ASSUNCAO 81609426134,1475.10,1475.10,7,103.2570,NaN,103.2570
4,ANDREIA,AMERIPEL CARLOS,2025-06-27,7340,PAPELARIA AVENIDA LTDA,2069.20,2069.20,7,144.8440,20.0,164.8440


### load(gera relatorios linha a linha e salva em todos os lugares) 09 - comissoes e resumo de comissoes 


In [4]:
path_relatorio = fr'g:\Meu Drive\MASQUERADE\FINANCEIRO\RELATORIOS\COMISSOES\06-2025 em diante\relatorio_comissao_0{mes}-0{mes2}_{ano}.xlsx'
workbook = xlsxwriter.Workbook(path_relatorio)
worksheet = workbook.add_worksheet('Relatório')

# Formatações
titulo = workbook.add_format({
    'bold': True, 
    'bg_color': "#A7A7A7", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
supervisor_format = workbook.add_format({
    'bold': True, 
    'bg_color': "#6680F0", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
vendedor_format = workbook.add_format({
    'bold': True, 
    'bg_color': "#60A165", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
cols = workbook.add_format({
    'border': 6, 
    'bg_color': "#92D397", 
    'align': 'center', 'valign': 'vcenter'
})
infos = workbook.add_format({
    'border': 2, 
    'align': 'center', 'valign': 'vcenter'
})
infos_num = workbook.add_format({
    'border': 2, 
    'align': 'center', 'valign': 'vcenter',
    'num_format': 'R$ #.##0,00'
})
worksheet.merge_range('A1:H1', f"relatorio de comissão do mes {mes}/{2025} para pagar no mes {mes2}/{2025}", titulo)
linha = 3

# Supervisores
for supervisor in df_com_supervisor['Nm supervisor'].unique():
    worksheet.merge_range(linha, 0, linha, 2,  f"SUPERVISOR: {supervisor}", supervisor_format)
    linha += 2

    vendedores = df_com_supervisor[df_com_supervisor['Nm supervisor'] == supervisor]['Nm Vendedor'].unique()
    for vendedor in vendedores:
        worksheet.write(linha, 2, f"VENDEDOR: {vendedor}", vendedor_format)
        linha += 1

        headers = ["Data emissão", "NF", "Cliente", "Venda", "Faturado", "Comissão", "Cliente NV", "Total"]
        for col, header in enumerate(headers, start=0):
            worksheet.write(linha, col, header, cols)
        linha += 1

        vendas = df_com_supervisor[
            (df_com_supervisor['Nm supervisor'] == supervisor) &
            (df_com_supervisor['Nm Vendedor'] == vendedor)
        ]
        colunas_numericas = [3, 4, 5, 6, 7]
        for _, row in vendas.iterrows():
            dados = [
                row['Dt. emissão'],
                f"NF {row['Nr. nota']}",
                row['Descrição'],
                row['Vl venda'],
                row['Vl Total Nota'],
                row['comissao'],
                row['cliente_nv'],
                row['total']
            ]
            for col, val in enumerate(dados, start=0):
                val = "" if pd.isna(val) else val
                estilo = infos_num if col in colunas_numericas else infos
                worksheet.write(linha, col, val, estilo)
            linha += 1

        subtot = subtotal_vendedor[
            (subtotal_vendedor['Nm supervisor'] == supervisor) &
            (subtotal_vendedor['Nm Vendedor'] == vendedor)
        ]
        if not subtot.empty:
            worksheet.write(linha, 2    , "TOTAL:", cols)
            worksheet.write(linha, 3, subtot['total_venda'].values[0], infos_num)
            worksheet.write(linha, 4, subtot['total_faturado'].values[0], infos_num)
            worksheet.write(linha, 5, subtot['total_comissao'].values[0], infos_num)
            worksheet.write(linha, 6, subtot['total_cliente_nv'].values[0], infos_num)
            worksheet.write(linha, 7, subtot['total_total'].values[0], infos_num)
            linha += 2
    resumo = resumo_supervisor.loc[supervisor]
    worksheet.write(linha, 2, f"→ Comissão cliente NV {supervisor}:", cols)
    worksheet.write(linha, 3, resumo['total_cliente_nv'], infos_num)
    worksheet.write(linha + 1, 2, f"→ 3% do grupo do {supervisor}:", cols)
    worksheet.write(linha + 1, 3, resumo['comissao_extra_3pct'], infos_num)
    worksheet.write(linha + 2, 2, f"→ TOTAL COMISSÃO FINAL {supervisor}:", cols)
    worksheet.write(linha + 2, 3, resumo['comissao_final'], infos_num)
    linha += 5

# Vendedores sem supervisor
for vendedor in df_sem_supervisor['Nm Vendedor'].unique():
    worksheet.write(linha, 2, f"VENDEDOR: {vendedor}", vendedor_format)
    linha += 1

    headers = ["Data emissão", "NF", "Cliente", "Venda", "Faturado", "Comissão", "Cliente NV", "Total"]
    for col, header in enumerate(headers, start=0):
        worksheet.write(linha, col, header, cols)
    linha += 1

    vendas = df_sem_supervisor[df_sem_supervisor['Nm Vendedor'] == vendedor]
    colunas_numericas = [3, 4, 5, 6, 7]
    for _, row in vendas.iterrows():
        dados = [
            row['Dt. emissão'],
            f"NF {row['Nr. nota']}",
            row['Descrição'],
            row['Vl venda'],
            row['Vl Total Nota'],
            row['comissao'],
            row['cliente_nv'],
            row['total']
        ]
        for col, val in enumerate(dados, start=0):
            val = "" if pd.isna(val) else val
            estilo = infos_num if col in colunas_numericas else infos
            worksheet.write(linha, col, val, estilo)
        linha += 1

    subtot = subtotal_vendedor[subtotal_vendedor['Nm Vendedor'] == vendedor]
    if not subtot.empty:
        worksheet.write(linha, 2, "TOTAL:", cols)
        worksheet.write(linha, 3, subtot['total_venda'].values[0], infos_num)
        worksheet.write(linha, 4, subtot['total_faturado'].values[0], infos_num)
        worksheet.write(linha, 5, subtot['total_comissao'].values[0], infos_num)
        worksheet.write(linha, 6, subtot['total_cliente_nv'].values[0], infos_num)
        worksheet.write(linha, 7, subtot['total_total'].values[0], infos_num)
        linha += 2

# Fecha corretamente o arquivo
workbook.close()


NameError: name 'xlsxwriter' is not defined

In [ ]:
# Garantir que 'Nm Vendedor' seja o index para facilitar
subtotal_vendedor = subtotal_vendedor.set_index('Nm Vendedor')

# Loop nos nomes do resumo_supervisor (supervisores)
for nome_supervisor in resumo_supervisor.index:
    if nome_supervisor in subtotal_vendedor.index:
        # Soma a comissão de supervisor no total do vendedor
        subtotal_vendedor.loc[nome_supervisor, 'total_comissao'] = resumo_supervisor.loc[nome_supervisor, 'comissao_final']
    else:
        # Se o supervisor não é vendedor, adiciona nova linha
        subtotal_vendedor.loc[nome_supervisor] = resumo_supervisor.loc[nome_supervisor, 'comissao_final']

# Resetar index se necessário
subtotal_vendedor = subtotal_vendedor.reset_index()

# resumo
path_resumo = fr'g:\Meu Drive\MASQUERADE\FINANCEIRO\RELATORIOS\COMISSOES\06-2025 em diante\resumo_comissao_0{mes}-0{mes2}_{ano}.xlsx'
workbook = xlsxwriter.Workbook(path_resumo)
worksheet = workbook.add_worksheet('Resumo')

linha = 0

titulo = workbook.add_format({
    'bold': True, 
    'bg_color': "#A7A7A7", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
cols = workbook.add_format({
    'border': 6, 
    'bold': True,
    'bg_color': "#72AC76", 
    'align': 'center', 'valign': 'vcenter'
})
infos = workbook.add_format({
    'border': 2, 
    'bg_color': "#94EEA3", 
    'align': 'center', 'valign': 'vcenter'
})
infos_num = workbook.add_format({
    'border': 2, 
    'bg_color': "#94EEA3", 
    'align': 'center', 'valign': 'vcenter',
        'num_format': 'R$ #.##0,00'
})


worksheet.merge_range('A1:G1', f'POSIÇÃO DE COMISSÕES DOS REPRESENTANTES 0{mes} PAGAMENTO EM 0{mes2}/{ano}', titulo)
linha += 1

headers = ['REPRESENTANTE',	'COMISSÃO',	'BANCO', 'AG.', 'CONTA', 'FAVORECIDO', 'CPF/CNPJ']
for col, header in enumerate(headers, start=0):
    worksheet.write(linha, col, header, cols)
    
linha += 1

for vendedor in subtotal_vendedor['Nm Vendedor'].unique():
    worksheet.write(linha, 0, vendedor, infos)
    worksheet.write(linha, 3, '', infos)
    worksheet.write(linha, 4, '', infos)
    worksheet.write(linha, 5, '', infos)
    worksheet.write(linha, 6, '', infos)
    worksheet.write(linha, 2, 'PIX', infos)

    valor = subtotal_vendedor.loc[
        subtotal_vendedor['Nm Vendedor'] == vendedor, 'total_comissao'
    ].values[0]

    worksheet.write(linha, 1, valor, infos_num)
    linha += 2
    
workbook.close()

